In [1]:
!nvidia-smi
# 63103MiB / 81920MiB -->starcoder utilization

Thu Mar 14 02:39:05 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:4D:00.0 Off |                    0 |
| N/A   35C    P0    64W / 400W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# internet allow way
import os
# run internet
os.environ['http_proxy']="http://proxy62.iitd.ac.in:3128"
os.environ['ftp_proxy']="http://proxy62.iitd.ac.in:3128"
os.environ['https_proxy']="http://proxy62.iitd.ac.in:3128"

In [4]:
!conda info


     active environment : plempirical
    active env location : /home/cse/dual/cs5190439/.conda/envs/plempirical
            shell level : 2
       user config file : /home/cse/dual/cs5190439/.condarc
 populated config files : 
          conda version : 4.9.2
    conda-build version : 3.17.8
         python version : 3.6.10.final.0
       virtual packages : __cuda=11.8=0
                          __glibc=2.17=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /home/apps/anaconda3_2018/4.6.9  (read only)
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /home/apps/anaconda3_2018/4.6.9/pkgs
                          /home/cse/dual/cs5190439/.conda/pkgs
       envs directories : /hom

In [5]:

import requests
try:
  # Check any accessible URL, like Google's homepage
  response = requests.get("https://www.google.com/")
  if response.status_code == 200:
    print("Internet connection active!")
  else:
    print("Internet connection available, but unable to reach the specified URL.")
except requests.exceptions.ConnectionError:
  print("No internet connection available.")

Internet connection active!


In [6]:
import os
import logging
from dotenv import load_dotenv
import time
import argparse
from tqdm import tqdm
import torch
# import codegeex
# from codegeex.tokenizer import CodeGeeXTokenizer
# from codegeex.torch import CodeGeeXModel
from transformers import AutoTokenizer, AutoModelForCausalLM


In [7]:
source_lang="Java"
target_lang="Python"
model_name= "StarCoder"

In [8]:
k=50
p=0.95
temperature= 0.2

In [9]:
# elif args.model == 'StarCoder':
# tokenizer = AutoTokenizer.from_pretrained('bigcode/starcoder', use_auth_token=os.environ['STARCODER_AUTH_TOKEN'], cache_dir='./huggingface')
# model = AutoModelForCausalLM.from_pretrained('bigcode/starcoder', use_auth_token=os.environ['STARCODER_AUTH_TOKEN'], cache_dir='./huggingface').to(device)
model_path="/scratch/scai/mtech/aib222684/starcoder"
output_dir="/scratch/cse/dual/cs5190439/MTP1/starcoder_data"
cache_dir="/scratch/cse/dual/cs5190439/MTP1/starcoder_data"

print("Loading the tokenizer")
tokenizer = AutoTokenizer.from_pretrained(model_path, use_auth_token=True, cache_dir=cache_dir)
gpu_id=0
device = f'cuda:{gpu_id}'
print("Loading the model")
model = AutoModelForCausalLM.from_pretrained(
model_path,
use_auth_token=True,cache_dir=cache_dir).to(device)

Loading the tokenizer


/home/cse/dual/cs5190439/.conda/envs/plempirical/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading the model


/home/cse/dual/cs5190439/.conda/envs/plempirical/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
# OPEN API GPT method for pseudocode based approach

In [11]:
import os
import openai
import logging
import tiktoken
from pathlib import Path
from dotenv import load_dotenv
import re
import argparse
from tqdm import tqdm

os.makedirs(f'logs', exist_ok=True)
logging.basicConfig(filename=f"logs/pseudocode.log", level=logging.INFO, format='%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')


In [18]:
load_dotenv()

# parser = argparse.ArgumentParser(description='run translation with GPT-4 with a given dataset and languages')
# parser.add_argument('--dataset', help='dataset to use for code translation. should be one of [codenet,avatar,evalplus]', required=True, type=str)
# parser.add_argument('--source_lang', help='source language to use for code translation. should be one of [Python,Java,C,C++,Go]', required=True, type=str)
# parser.add_argument('--target_lang', help='target language to use for code translation. should be one of [Python,Java,C,C++,Go]', required=True, type=str)
# parser.add_argument('--k', help='The number of highest probability vocabulary tokens to keep for top-k-filtering. Only applies for sampling mode, with range from 1 to 100.', required=True, type=int)
# parser.add_argument('--p', help='Only the most probable tokens with probabilities that add up to top_p or higher are considered during decoding. The valid range is 0.0 to 1.0. 1.0 is equivalent to disabled and is the default. Only applies to sampling mode. Also known as nucleus sampling.', required=True, type=float)
# parser.add_argument('--temperature', help='A value used to warp next-token probabilities in sampling mode. Values less than 1.0 sharpen the probability distribution, resulting in "less random" output. Values greater than 1.0 flatten the probability distribution, resulting in "more random" output. A value of 1.0 has no effect and is the default. The allowed range is 0.0 to 2.0.', required=True, type=float)
# args = parser.parse_args()
dataset="avatar"
source_lang ="Java"
target_lang="Python"
source = source_lang
target = target_lang
notation="Control Structure Notation"
k=50
p=0.95
temperature=0.7

In [20]:
import os
class Translate:
    EXTENSTIONS = {
        "Java": "java",
        "Python": "py",
        "Go": "go",
        "C": "c",
        "C++": "cpp",
        "Rust": "rs",
        "C#": "cs"
    }

    def __init__(self, dataset) -> None:
        # Set up OpenAI API key
        # self.model = 'GPT-4'
        self.model="gpt-3.5-turbo"
        self.dataset = dataset

    def __enter__(self):
        api_key = os.getenv("OPENAI_API_KEY")
        openai.api_key = api_key
        # openai.api_key = "aacf0adc15ef4bcd9c1c93e9f2f19134"
        # openai.api_base = "https://openai-cs1200869.openai.azure.com/" 
        # openai.api_type = 'azure'
        # openai.api_version = '2023-05-15'
        
        logging.info(f"successfully set up openai api key")

        # self.main_dir = os.getcwd()
        self.main_dir= "/scratch/cse/dual/cs5190439/MTP1/PLTranslation_data"
        # see here
        self.output_dir = os.path.join(self.main_dir, "pseudocode")

        # Find the input directory with all the code examples
        self.input_dir = Path(self.main_dir).joinpath("dataset", self.dataset)

        self.hf_cache_dir = os.path.join(self.main_dir, "hf_cache_dir_pseudocode")

        if not self.input_dir.exists():
            logging.error(f"directory {str(self.input_dir)} does not exist. raising FileNotFoundError")
            raise FileNotFoundError(f"Directory {str(self.input_dir)} does not exist.")

        self.out_dir = Path(self.output_dir).joinpath(self.model, self.dataset)
        if not self.out_dir.exists():
            self.out_dir.mkdir(parents=True)

        return self
    
    # def find_file_with_smallest_length(self,folder_path, prefix):ignoring it this time


    def send_message_to_openai(self, message_log):
        # "Use OpenAI's ChatCompletion API to get the chatbot's response"
        # print("Entered send_message_to_openai")
        
        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
        num_tokens = len(encoding.encode(message_log[1]["content"]))

        response = "exceptional case"
        is_success = False
        max_attempts = 5
        while max_attempts > 0:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",  # The name of the OpenAI chatbot model to use
                    # engine='gpt-35-turbo',
                    # The conversation history up to this point, as a list of dictionaries
                    messages=message_log,
                    # The maximum number of tokens (words or subwords) in the generated response
                    max_tokens=max(1, 4000-num_tokens),
                    # The "creativity" of the generated response (higher temperature = more creative)
                    temperature=0.7,
                )
                is_success = True
                break
            except openai.error.InvalidRequestError as e:
                print("# Token size exceeded.- error")
                return "# Token size exceeded."
            except:
                max_attempts -= 1
                continue

        if not is_success:
            print(response,"-error")
            return response

        # Find the first response from the chatbot that has text in it (some responses may not have text)
        for choice in response.choices:
            if "text" in choice:
                return choice.text

        # If no response with text is found, return the first response's content (which may be empty)
        return response.choices[0].message.content

    def translate_with_OPENAI(self, source, code_as_str, to, notation="Control Structure Notation"):
        # content = code_as_str + f"\n# Translate the above {source} code to {to}. Print only the {to} code and end with the comment \"End of Code\".\n"
        content = code_as_str + f"\n# Write pseudocode in '{notation}' for the above {source} code. Print only this pseudocode and end with the comment '# End of Pseudocode'."
        message = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": content}]
        response = self.send_message_to_openai(message)
        # return response.replace(f"```{to.lower()}", "").replace("```", "")
        return response

    def translate(self, source, target, notation="Control Structure Notation"):
        snippets = list(self.input_dir.joinpath(str(source), "Code").iterdir())
        
        # exp_try
        # snippets=snippets[:2]
        # print(snippets)

        for source_file in tqdm(snippets, total=len(snippets), bar_format="{desc:<5.5}{percentage:3.0f}%|{bar:10}{r_bar}"):
            code_id = source_file.stem
            code_as_str = source_file.read_text(encoding="utf-8")
            # print(source_file)

            target_dir = self.out_dir.joinpath(f"{source}", f"{target}")
            if not target_dir.exists():
                target_dir.mkdir(parents=True)

            filename_of_translated_code = target_dir.joinpath(f"{code_id}.{Translate.EXTENSTIONS[source]}")

            translated_code_fp = Path(filename_of_translated_code)
            # if translated_code_fp.exists():
            #     continue

            translated_code = self.translate_with_OPENAI(source, code_as_str, target, notation)
            # translated_code = re.sub('public\s*class\s*.+', 'public class ' + code_id + ' {', translated_code)

            if self.dataset == 'evalplus' and target == 'Java':
                translated_code = 'package com.example;\n' + translated_code #ignore

            with open(filename_of_translated_code, "w") as f:
                print(translated_code, file=f)
                    
    def __exit__(self, exception, _, __):
        print(exception)


In [15]:
# translate_with_OPENAI(source, code1, target)

NameError: name 'translate_with_OPENAI' is not defined

In [21]:
# for all files
notation="Control Structure Notation"
with Translate(dataset) as translator:
    logging.info(f"writing pseudocode of examples in {source} using GPT and {dataset} dataset")
    translator.translate(source, target, notation)


     100%|██████████| 249/249 [13:17<00:00,  3.20s/it]

None


In [19]:
code1='''
public class Main {
    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);
        String S = scanner.nextLine();
        String T = scanner.nextLine();
        int min = Integer.MAX_VALUE;
        for(int i = 0; i<=S.length()-T.length(); i++){
            min = Math.min(min, find(S.substring(i, i+T.length()), T));
        }
        if(min!=Integer.MAX_VALUE){
            System.out.println(min);
        }else{
            System.out.println(T.length());
        }

    }

    public static int find(String a, String b){
        int calc = 0;
        for(int x = 0; x<a.length(); x++){
            if(a.charAt(x)!= b.charAt(x)){
                calc++;
            }
        }
        return calc;
    }
}
'''

In [44]:
code_exp1='''```
Start
    Read input string S
    Read input string T
    Set min to Integer.MAX_VALUE
    Loop i from 0 to (S.length() - T.length()):
        Set substring A as S.substring(i, i + T.length())
        Set min to minimum of min and find(A, T)
    If min is not equal to Integer.MAX_VALUE:
        Print min
    Else:
        Print length of T

Function find(a, b):
    Set calc to 0
    Loop x over the length of a:
        If a[x] is not equal to b[x]:
            Increment calc
    Return calc
End
```
'''

In [24]:
source_code=code1
code_exp=code_exp1
# review
# prompt = f"Translate below pseudocode algorithm in '{notation}' to {args.target_lang} code. Print only the {args.target_lang} code and end with the comment \"End of Code\".\n\npseudocode algorithm:\n\n{prompt_exp}\n\n{args.target_lang} Code:\n\n"
prompt = f"Translate below pseudocode in '{notation}' code to {target_lang} code. Print only the {target_lang} code and end with the comment 'End of Code'.\n\npseudocode:\n\n{code_exp}\n\n{target_lang} Code:\n\n"
prefix_token = "<fim_prefix>"
suffix_token = "<fim_suffix><fim_middle>"
prompt = prefix_token + prompt + suffix_token
# Your generated {target} code should

In [25]:
try:
    t0 = time.perf_counter()
    inputs=[]
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    total_input_tokens = inputs.shape[1]
    model_max_length = 4096 if model_name == 'LLaMa' else 2048
    if total_input_tokens >= model_max_length:
        print("# Token size exceeded.")
        # continue
    max_new_tokens = model_max_length - total_input_tokens
    raw_outputs = ''
    raw_outputs = model.generate(
    inputs,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    top_p=p,
    top_k=k,
    temperature=temperature,
    repetition_penalty=1,
    pad_token_id=tokenizer.eos_token_id,
    )
    t1 = time.perf_counter()
    print("Total generation time:", t1 - t0)
    print(tokenizer.decode(raw_outputs[0]))
except (ValueError, FileNotFoundError) as e:
    print(e)
    # continue

Total generation time: 18.0958413798362
<fim_prefix>Translate below pseudocode in Control Structure Notation code to Python code. Print only the Python code and end with the comment "End of Code".

pseudocode:


Start
    Read input string S
    Read input string T
    Set min to Integer.MAX_VALUE
    Loop i from 0 to (S.length() - T.length()):
        Set substring A as S.substring(i, i + T.length())
        Set min to minimum of min and find(A, T)
    If min is not equal to Integer.MAX_VALUE:
        Print min
    Else:
        Print length of T

Function find(a, b):
    Set calc to 0
    Loop x over the length of a:
        If a[x] is not equal to b[x]:
            Increment calc
    Return calc
End


Python Code:

<fim_suffix><fim_middle>Start
    S = input("Enter string S: ")
    T = input("Enter string T: ")
    min = sys.maxsize
    for i in range(len(S) - len(T)):
        A = S[i : i + len(T)]
        if min > find(A, T):
            min = find(A, T)
    if min is not sys.maxsi

In [26]:
source_code=code1
code_exp=code_exp1
# review
prompt = f"Translate below {source_lang} code to {target_lang} code by considering below {source_lang} code and its pseudocode. Print only the {target_lang} code and end with the comment \"End of Code\".\n\n{source_lang} Code:\n\n{source_code}\n\npseudocode:\n\n{code_exp}\n\n{target_lang} Code:\n\n"
prefix_token = "<fim_prefix>"
suffix_token = "<fim_suffix><fim_middle>"
prompt = prefix_token + prompt + suffix_token
# Your generated {target} code should

In [27]:
try:
    t0 = time.perf_counter()
    inputs=[]
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    total_input_tokens = inputs.shape[1]
    model_max_length = 4096 if model_name == 'LLaMa' else 2048
    if total_input_tokens >= model_max_length:
        print("# Token size exceeded.")
        # continue
    max_new_tokens = model_max_length - total_input_tokens
    raw_outputs = ''
    raw_outputs = model.generate(
    inputs,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    top_p=p,
    top_k=k,
    temperature=temperature,
    repetition_penalty=1,
    pad_token_id=tokenizer.eos_token_id,
    )
    t1 = time.perf_counter()
    print("Total generation time:", t1 - t0)
    print(tokenizer.decode(raw_outputs[0]))
except (ValueError, FileNotFoundError) as e:
    print(e)
    # continue

Total generation time: 8.065612243488431
<fim_prefix>Translate below Java code to Python code by considering below Java code and its pseudocode notation. Print only the Python code and end with the comment "End of Code".

Java Code:


public class Main {
    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);
        String S = scanner.nextLine();
        String T = scanner.nextLine();
        int min = Integer.MAX_VALUE;
        for(int i = 0; i<=S.length()-T.length(); i++){
            min = Math.min(min, find(S.substring(i, i+T.length()), T));
        }
        if(min!=Integer.MAX_VALUE){
            System.out.println(min);
        }else{
            System.out.println(T.length());
        }

    }

    public static int find(String a, String b){
        int calc = 0;
        for(int x = 0; x<a.length(); x++){
            if(a.charAt(x)!= b.charAt(x)){
                calc++;
            }
        }
        return calc;
    }
}


pseudocode:



In [21]:
code2='''import java.util.Scanner ; public class codeforces_242_A { public static void main ( String [ ] args ) { Scanner in = new Scanner ( System.in ) ; int x , y , a , b ; x = in.nextInt ( ) ; y = in.nextInt ( ) ; a = in.nextInt ( ) ; b = in.nextInt ( ) ; int count = 0 ; for ( int i = a ; i <= x ; i ++ ) { for ( int j = b ; j <= y ; ++ j ) { if ( i <= j ) continue ; count ++ ; } } System.out.println ( count ) ; for ( int i = a ; i <= x ; i ++ ) { for ( int j = b ; j <= y ; ++ j ) { if ( i <= j ) continue ; System.out.println ( i + " " + j ) ; } } in.close ( ) ; } }'''

In [49]:
code_exp2='''
Start
    Input x, y, a, b
    Set count to 0
    For i from a to x
        For j from b to y
            If i is less than or equal to j
                Continue
            Increment count
    Print count
    For i from a to x
        For j from b to y
            If i is less than or equal to j
                Continue
            Print i and j
    Close input scanner
End

# End of Pseudocode
'''

In [31]:
source_code=code2
code_exp=code_exp2
# review
prompt = f"Translate below pseudocode in '{notation}' code to {target_lang} code. Print only the {target_lang} code and end with the comment 'End of Code'.\n\npseudocode:\n\n{code_exp}\n\n{target_lang} Code:\n\n"
prefix_token = "<fim_prefix>"
suffix_token = "<fim_suffix><fim_middle>"
prompt = prefix_token + prompt + suffix_token
# Your generated {target} code should

In [32]:
try:
    t0 = time.perf_counter()
    inputs=[]
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    total_input_tokens = inputs.shape[1]
    model_max_length = 4096 if model_name == 'LLaMa' else 2048
    if total_input_tokens >= model_max_length:
        print("# Token size exceeded.")
        # continue
    max_new_tokens = model_max_length - total_input_tokens
    raw_outputs = ''
    raw_outputs = model.generate(
    inputs,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    top_p=p,
    top_k=k,
    temperature=temperature,
    repetition_penalty=1,
    pad_token_id=tokenizer.eos_token_id,
    )
    t1 = time.perf_counter()
    print("Total generation time:", t1 - t0)
    print(tokenizer.decode(raw_outputs[0]))
except (ValueError, FileNotFoundError) as e:
    print(e)
    # continue

Total generation time: 5.631102414801717
<fim_prefix>Translate below pseudocode in 'Control Structure Notation' code to Python code. Print only the Python code and end with the comment 'End of Code'.

pseudocode:

Start
    Input x, y, a, b
    Set count to 0
    For i from a to x
        For j from b to y
            If i is less than or equal to j
                Continue
            Increment count
    Print count
    For i from a to x
        For j from b to y
            If i is less than or equal to j
                Continue
            Print i and j
    Close input scanner
End

# End of Pseudocode


Python Code:

<fim_suffix><fim_middle>
x = input('Enter value of x: ')
y = input('Enter value of y: ')
a = input('Enter value of a: ')
b = input('Enter value of b: ')
count = 0
for i in range(a, x):
    for j in range(b, y):
        if i <= j:
            continue
        count += 1
print(count)
for i in range(a, x):
    for j in range(b, y):
        if i <= j:
            continue


In [47]:
source_code=code1
code_exp=code_exp1
# review
prompt = f"Translate below {source_lang} code to {target_lang} code by considering below {source_lang} code and its pseudocode. Print only the {target_lang} code and end with the comment \"End of Code\".\n\n{source_lang} Code:\n\n{source_code}\n\npseudocode:\n\n{code_exp}\n\n{target_lang} Code:\n\n"
prefix_token = "<fim_prefix>"
suffix_token = "<fim_suffix><fim_middle>"
prompt = prefix_token + prompt + suffix_token
# Your generated {target} code should

In [48]:
try:
    t0 = time.perf_counter()
    inputs=[]
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    total_input_tokens = inputs.shape[1]
    model_max_length = 4096 if model_name == 'LLaMa' else 2048
    if total_input_tokens >= model_max_length:
        print("# Token size exceeded.")
        # continue
    max_new_tokens = model_max_length - total_input_tokens
    raw_outputs = ''
    raw_outputs = model.generate(
    inputs,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    top_p=p,
    top_k=k,
    temperature=temperature,
    repetition_penalty=1,
    pad_token_id=tokenizer.eos_token_id,
    )
    t1 = time.perf_counter()
    print("Total generation time:", t1 - t0)
    print(tokenizer.decode(raw_outputs[0]))
except (ValueError, FileNotFoundError) as e:
    print(e)
    # continue

Total generation time: 7.18512009177357
<fim_prefix>Translate below Java code to Python code by considering below Java code and its pseudocode. Print only the Python code and end with the comment "End of Code".

Java Code:


public class Main {
    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);
        String S = scanner.nextLine();
        String T = scanner.nextLine();
        int min = Integer.MAX_VALUE;
        for(int i = 0; i<=S.length()-T.length(); i++){
            min = Math.min(min, find(S.substring(i, i+T.length()), T));
        }
        if(min!=Integer.MAX_VALUE){
            System.out.println(min);
        }else{
            System.out.println(T.length());
        }

    }

    public static int find(String a, String b){
        int calc = 0;
        for(int x = 0; x<a.length(); x++){
            if(a.charAt(x)!= b.charAt(x)){
                calc++;
            }
        }
        return calc;
    }
}


pseudocode:

```
Start


In [ ]:
# problem in code+pseudocode approach
# -check prev

In [54]:
"'''"=="```"

False

In [56]:
source_code=code2
code_exp=code_exp2
prompt = f"Translate below {source_lang} code to {target_lang} code by considering below {source_lang} code and its pseudocode algorithm. Print only the {target_lang} code and end with the comment \"End of Code\".\n\n{source_lang} Code:\n\n{source_code}\n\npseudocode algorithm:\n\n{code_exp}\n\n{target_lang} Code:\n\n"
prefix_token = "<fim_prefix>"
suffix_token = "<fim_suffix><fim_middle>"
prompt = prefix_token + prompt + suffix_token
# Your generated {target} code should

In [57]:
try:
    t0 = time.perf_counter()
    inputs=[]
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    total_input_tokens = inputs.shape[1]
    model_max_length = 4096 if model_name == 'LLaMa' else 2048
    if total_input_tokens >= model_max_length:
        print("# Token size exceeded.")
        # continue
    max_new_tokens = model_max_length - total_input_tokens
    raw_outputs = ''
    raw_outputs = model.generate(
    inputs,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    top_p=p,
    top_k=k,
    temperature=temperature,
    repetition_penalty=1,
    pad_token_id=tokenizer.eos_token_id,
    )
    t1 = time.perf_counter()
    print("Total generation time:", t1 - t0)
    print(tokenizer.decode(raw_outputs[0]))
except (ValueError, FileNotFoundError) as e:
    print(e)
    # continue

Total generation time: 5.430065448395908
<fim_prefix>Translate below Java code to Python code by considering below Java code and its pseudocode algorithm. Print only the Python code and end with the comment "End of Code".

Java Code:

import java.util.Scanner ; public class codeforces_242_A { public static void main ( String [ ] args ) { Scanner in = new Scanner ( System.in ) ; int x, y, a, b ; x = in.nextInt ( ) ; y = in.nextInt ( ) ; a = in.nextInt ( ) ; b = in.nextInt ( ) ; int count = 0 ; for ( int i = a ; i <= x ; i ++ ) { for ( int j = b ; j <= y ; ++ j ) { if ( i <= j ) continue ; count ++ ; } } System.out.println ( count ) ; for ( int i = a ; i <= x ; i ++ ) { for ( int j = b ; j <= y ; ++ j ) { if ( i <= j ) continue ; System.out.println ( i + " " + j ) ; } } in.close ( ) ; } }

pseudocode algorithm:


Start
    Input x, y, a, b
    Set count to 0
    For i from a to x
        For j from b to y
            If i is less than or equal to j
                Continue
            In

In [27]:
code3='''import java.util.Scanner ; import java.io.BufferedReader ; import java.io.IOException ; import java.io.InputStreamReader ; import java.io.PrintWriter ; import java.nio.charset.MalformedInputException ; import java.util.StringTokenizer ; import java.util.Arrays ; import java.util.HashSet ; public class codeforces_608_A { public static void main ( String [ ] args ) { FastScanner in = new FastScanner ( ) ; Scanner sc = new Scanner ( System.in ) ; PrintWriter o = new PrintWriter ( System.out ) ; int n = sc.nextInt ( ) ; int s = sc.nextInt ( ) ; int max = 0 ; while ( n -- > 0 ) { int f = sc.nextInt ( ) ; int t = sc.nextInt ( ) ; if ( max < f + t ) { max = f + t ; } } o.println ( Math.max ( max, s ) ) ; o.close ( ) ; } static class FastScanner { BufferedReader br = new BufferedReader ( new InputStreamReader ( System.in ) ) ; StringTokenizer st = new StringTokenizer ( "" ) ; public String next ( ) { while (! st.hasMoreElements ( ) ) { try { st = new StringTokenizer ( br.readLine ( ) ) ; } catch ( IOException e ) { e.printStackTrace ( ) ; } } return st.nextToken ( ) ; } public String nextLine ( ) { String str = "" ; try { str = br.readLine ( ) ; } catch ( IOException e ) { e.printStackTrace ( ) ; } return str ; } byte nextByte ( ) { return Byte.parseByte ( next ( ) ) ; } short nextShort ( ) { return Short.parseShort ( next ( ) ) ; } int nextInt ( ) { return Integer.parseInt ( next ( ) ) ; } long nextLong ( ) { return java.lang.Long.parseLong ( next ( ) ) ; } double nextDouble ( ) { return Double.parseDouble ( next ( ) ) ; } } }'''

In [35]:
code_exp3='''
Start
  Initialize n as integer
  Initialize s as integer
  Initialize max as integer
  Read n from input
  Read s from input
  while n > 0
    Read f from input
    Read t from input
    if max < f + t
      max = f + t
    end if
    Decrement n by 1
  end while
  Print the maximum value between max and s
End

# End of Pseudocode
'''

In [58]:
source_code=code3
code_exp=code_exp3
prompt = f"Translate below {source_lang} code to {target_lang} code by considering below {source_lang} code and its pseudocode algorithm. Print only the {target_lang} code and end with the comment \"End of Code\".\n\n{source_lang} Code:\n\n{source_code}\n\npseudocode algorithm:\n\n{code_exp}\n\n{target_lang} Code:\n\n"
prefix_token = "<fim_prefix>"
suffix_token = "<fim_suffix><fim_middle>"
prompt = prefix_token + prompt + suffix_token
# Your generated {target} code should

In [59]:
try:
    t0 = time.perf_counter()
    inputs=[]
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    total_input_tokens = inputs.shape[1]
    model_max_length = 4096 if model_name == 'LLaMa' else 2048
    if total_input_tokens >= model_max_length:
        print("# Token size exceeded.")
        # continue
    max_new_tokens = model_max_length - total_input_tokens
    raw_outputs = ''
    raw_outputs = model.generate(
    inputs,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    top_p=p,
    top_k=k,
    temperature=temperature,
    repetition_penalty=1,
    pad_token_id=tokenizer.eos_token_id,
    )
    t1 = time.perf_counter()
    print("Total generation time:", t1 - t0)
    print(tokenizer.decode(raw_outputs[0]))
except (ValueError, FileNotFoundError) as e:
    print(e)
    # continue

Total generation time: 12.726417613215744
<fim_prefix>Translate below Java code to Python code by considering below Java code and its pseudocode algorithm. Print only the Python code and end with the comment "End of Code".

Java Code:

import java.util.Scanner ; import java.io.BufferedReader ; import java.io.IOException ; import java.io.InputStreamReader ; import java.io.PrintWriter ; import java.nio.charset.MalformedInputException ; import java.util.StringTokenizer ; import java.util.Arrays ; import java.util.HashSet ; public class codeforces_608_A { public static void main ( String [ ] args ) { FastScanner in = new FastScanner ( ) ; Scanner sc = new Scanner ( System.in ) ; PrintWriter o = new PrintWriter ( System.out ) ; int n = sc.nextInt ( ) ; int s = sc.nextInt ( ) ; int max = 0 ; while ( n -- > 0 ) { int f = sc.nextInt ( ) ; int t = sc.nextInt ( ) ; if ( max < f + t ) { max = f + t ; } } o.println ( Math.max ( max, s ) ) ; o.close ( ) ; } static class FastScanner { BufferedReader

In [31]:
code4='''import java.util.Scanner ; public class atcoder_ARC102_C { static int mod = 998244353 ; static long [ ] [ ] nchoosek ; public static void main ( String [ ] args ) { nchoosek = new long [ 4001 ] [ 4001 ] ; for ( int i = 0 ; i < nchoosek.length ; i ++ ) { nchoosek [ i ] [ 0 ] = nchoosek [ i ] [ i ] = 1 ; } for ( int i = 2 ; i < nchoosek.length ; i ++ ) { for ( int j = 1 ; j < i ; j ++ ) { nchoosek [ i ] [ j ] = ( nchoosek [ i - 1 ] [ j ] + nchoosek [ i - 1 ] [ j - 1 ] ) % mod ; } } Scanner sc = new Scanner ( System.in ) ; int k = sc.nextInt ( ) ; int n = sc.nextInt ( ) ; for ( int i = 2 ; i <= 2 * k ; i ++ ) { int pairs ; if ( i > k ) { pairs = k - i / 2 ; } else { pairs = ( i - 1 ) / 2 ; } int active = k - 2 * pairs ; if ( i % 2 == 0 ) active -= 1 ; long times2 = 1 ; long total = 0 ; for ( int j = 0 ; j <= pairs ; j ++ ) { long choice = times2 * nchoosek [ pairs ] [ j ] % mod ; times2 = times2 * 2 % mod ; if ( active + j - 1 < 0 ) continue ; total += choice * nchoosek [ n + active - 1 ] [ active + j - 1 ] ; if ( i % 2 == 0 ) { total += choice * nchoosek [ n + active - 2 ] [ active + j - 1 ] ; } total %= mod ; } System.out.println ( total ) ; } } }'''

In [38]:
code_exp4='''
start
    Initialize nchoosek array with size 4001x4001
    Loop i from 0 to nchoosek.length
        Set nchoosek[i][0] to 1
        Set nchoosek[i][i] to 1
    End Loop
    Loop i from 2 to nchoosek.length
        Loop j from 1 to i
            Set nchoosek[i][j] to (nchoosek[i-1][j] + nchoosek[i-1][j-1]) % mod
        End Loop
    End Loop
    Create a Scanner object named'sc' for input
    Read integer k from input
    Read integer n from input
    Loop i from 2 to 2*k
        If i > k
            Set pairs to k - i/2
        Else
            Set pairs to (i-1)/2
        End If
        Set active to k - 2*pairs
        If i is even
            Decrease active by 1
        End If
        Set times2 to 1
        Set total to 0
        Loop j from 0 to pairs
            Set choice to times2 * nchoosek[pairs][j] % mod
            Set times2 to times2 * 2 % mod
            If active + j - 1 < 0, continue to next iteration
            Add choice * nchoosek[n + active - 1][active + j - 1] to total
            If i is even
                Add choice * nchoosek[n + active - 2][active + j - 1] to total
            End If
            Set total to total % mod
        End Loop
        Print total
    End Loop
End
# End of Pseudocode
'''

In [60]:
source_code=code4
code_exp=code_exp4
prompt = f"Translate below {source_lang} code to {target_lang} code by considering below {source_lang} code and its pseudocode algorithm. Print only the {target_lang} code and end with the comment \"End of Code\".\n\n{source_lang} Code:\n\n{source_code}\n\npseudocode algorithm:\n\n{code_exp}\n\n{target_lang} Code:\n\n"
prefix_token = "<fim_prefix>"
suffix_token = "<fim_suffix><fim_middle>"
prompt = prefix_token + prompt + suffix_token
# Your generated {target} code should

In [61]:
try:
    t0 = time.perf_counter()
    inputs=[]
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    total_input_tokens = inputs.shape[1]
    model_max_length = 4096 if model_name == 'LLaMa' else 2048
    if total_input_tokens >= model_max_length:
        print("# Token size exceeded.")
        # continue
    max_new_tokens = model_max_length - total_input_tokens
    raw_outputs = ''
    raw_outputs = model.generate(
    inputs,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    top_p=p,
    top_k=k,
    temperature=temperature,
    repetition_penalty=1,
    pad_token_id=tokenizer.eos_token_id,
    )
    t1 = time.perf_counter()
    print("Total generation time:", t1 - t0)
    print(tokenizer.decode(raw_outputs[0]))
except (ValueError, FileNotFoundError) as e:
    print(e)
    # continue

Total generation time: 16.598990565165877
<fim_prefix>Translate below Java code to Python code by considering below Java code and its pseudocode algorithm. Print only the Python code and end with the comment "End of Code".

Java Code:

import java.util.Scanner ; public class atcoder_ARC102_C { static int mod = 998244353 ; static long [ ] [ ] nchoosek ; public static void main ( String [ ] args ) { nchoosek = new long [ 4001 ] [ 4001 ] ; for ( int i = 0 ; i < nchoosek.length ; i ++ ) { nchoosek [ i ] [ 0 ] = nchoosek [ i ] [ i ] = 1 ; } for ( int i = 2 ; i < nchoosek.length ; i ++ ) { for ( int j = 1 ; j < i ; j ++ ) { nchoosek [ i ] [ j ] = ( nchoosek [ i - 1 ] [ j ] + nchoosek [ i - 1 ] [ j - 1 ] ) % mod ; } } Scanner sc = new Scanner ( System.in ) ; int k = sc.nextInt ( ) ; int n = sc.nextInt ( ) ; for ( int i = 2 ; i <= 2 * k ; i ++ ) { int pairs ; if ( i > k ) { pairs = k - i / 2 ; } else { pairs = ( i - 1 ) / 2 ; } int active = k - 2 * pairs ; if ( i % 2 == 0 ) active -= 1 ; long 